In [6]:
import nfl

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import requests
from bs4 import BeautifulSoup as soup

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# import seaborn as sns
# sns.set()

In [23]:
df = nfl.load_data(2022)
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


https://en.wikipedia.org/wiki/Timeline_of_the_National_Football_League#The_1980s_and_1990s

In [108]:
#how many wins in a season does a team T have?
# open games.csv
games = pd.read_csv('games.csv')
games.head()
cols = ['game_id', 'season', 'game_type', 'week', 'home_team', 'away_team', 'home_score', 'away_score', 'result', 'total', 'overtime']
games = games[cols]
games.head()
#

,game_id,season,game_type,week,home_team,away_team,home_score,away_score,result,total,overtime
0,1999_01_MIN_ATL,1999,REG,1,ATL,MIN,14,17,-3,31,0
1,1999_01_KC_CHI,1999,REG,1,CHI,KC,20,17,3,37,0
2,1999_01_PIT_CLE,1999,REG,1,CLE,PIT,0,43,-43,43,0
3,1999_01_OAK_GB,1999,REG,1,GB,OAK,28,24,4,52,0
4,1999_01_BUF_IND,1999,REG,1,IND,BUF,31,14,17,45,0


In [109]:
def transform(row):
    if row['result'] > 0:
        row['home_team_record'] = 1
        row['away_team_record'] = 0
    elif row['result'] < 0:
        row['home_team_record'] = 0
        row['away_team_record'] = 1
    else:
        row['home_team_record'] = 0.5
        row['away_team_record'] = 0.5
    return row

games['home_team_record'] = 0
games['away_team_record'] = 0
# apply the function to each row of the DataFrame
games1 = games.apply(transform, axis=1)
games1.head()

,game_id,season,game_type,week,home_team,away_team,home_score,away_score,result,total,overtime,home_team_record,away_team_record
0,1999_01_MIN_ATL,1999,REG,1,ATL,MIN,14,17,-3,31,0,0.0,1.0
1,1999_01_KC_CHI,1999,REG,1,CHI,KC,20,17,3,37,0,1.0,0.0
2,1999_01_PIT_CLE,1999,REG,1,CLE,PIT,0,43,-43,43,0,0.0,1.0
3,1999_01_OAK_GB,1999,REG,1,GB,OAK,28,24,4,52,0,1.0,0.0
4,1999_01_BUF_IND,1999,REG,1,IND,BUF,31,14,17,45,0,1.0,0.0


In [112]:
g = games[games['season'] ==1999]
sorted(g['home_team'].unique())

['ARI',
 'ATL',
 'BAL',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GB',
 'IND',
 'JAX',
 'KC',
 'MIA',
 'MIN',
 'NE',
 'NO',
 'NYG',
 'NYJ',
 'OAK',
 'PHI',
 'PIT',
 'SD',
 'SEA',
 'SF',
 'STL',
 'TB',
 'TEN',
 'WAS']

In [24]:
#find how many games were played in the 2022 season
games = df['game_id'].unique().shape[0]
games = df['game_id'].unique().size
games

284

In [25]:
# how many weeks
num_weeks = df.week.unique().size
num_weeks
# data.week.unique()

22

In [26]:
#list games by week
g = df.groupby(['week'])
g[['game_id']].count()

,game_id
week,
1,2893
2,2779
3,2853
4,2828
5,2834
6,2507
7,2419
8,2618
9,2276


In [30]:
nodups = df.drop_duplicates(subset='game_id')
g = nodups.groupby('week')
g['game_id'].count()

week
1     16
2     16
3     16
4     16
5     16
6     14
7     14
8     15
9     13
10    14
11    14
12    16
13    15
14    13
15    16
16    16
17    15
18    16
19     6
20     4
21     2
22     1
Name: game_id, dtype: int64

In [32]:
for name, group in g['game_id']:
    print(f"Group '{name}':")
    print(group)

Group '1':
0       2022_01_BAL_NYJ
179      2022_01_BUF_LA
341     2022_01_CLE_CAR
527     2022_01_DEN_SEA
689      2022_01_GB_MIN
854     2022_01_IND_HOU
1074    2022_01_JAX_WAS
1264     2022_01_KC_ARI
1439     2022_01_LV_LAC
1604     2022_01_NE_MIA
1758     2022_01_NO_ATL
1949    2022_01_NYG_TEN
2121    2022_01_PHI_DET
2321    2022_01_PIT_CIN
2538     2022_01_SF_CHI
2719     2022_01_TB_DAL
Name: game_id, dtype: object
Group '2':
2893     2022_02_ARI_LV
3094     2022_02_ATL_LA
3259    2022_02_CAR_NYG
3435     2022_02_CHI_GB
3584    2022_02_CIN_DAL
3762    2022_02_HOU_DEN
3944    2022_02_IND_JAX
4102     2022_02_LAC_KC
4278    2022_02_MIA_BAL
4454    2022_02_MIN_PHI
4625     2022_02_NE_PIT
4795    2022_02_NYJ_CLE
4981     2022_02_SEA_SF
5145      2022_02_TB_NO
5323    2022_02_TEN_BUF
5491    2022_02_WAS_DET
Name: game_id, dtype: object
Group '3':
5672    2022_03_ATL_SEA
5839     2022_03_BAL_NE
6008    2022_03_BUF_MIA
6185    2022_03_CIN_NYJ
6375    2022_03_DAL_NYG
6559    2022_03_DET_M

In [42]:
#list all the teams
num_teams = df.home_team.unique().size
teams = df.home_team.unique()
teams = sorted(teams)
teams

['ARI',
 'ATL',
 'BAL',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GB',
 'HOU',
 'IND',
 'JAX',
 'KC',
 'LA',
 'LAC',
 'LV',
 'MIA',
 'MIN',
 'NE',
 'NO',
 'NYG',
 'NYJ',
 'PHI',
 'PIT',
 'SEA',
 'SF',
 'TB',
 'TEN',
 'WAS']

In [43]:
#abbreviations
teams = {
    'ARI': 'Arizona Cardinals',
    'ATL': 'Atlanta Falcons',
    'BAL':'Baltimore Ravens',
    'BUF':'Buffalo Bills',
    'CAR':'Carolina Panthers',
    'CHI':'Chicago Bears',
    'CIN':'Cincinnati Bengals',
    'CLE':'Cleveland Browns',
    'DAL':'Dallas Cowboys',
    'DEN':'Denver Broncos',
    'DET':'Detroit Lions',
    'GB':'Green Bay Packers',
    'HOU':'Houston Texans',
    'IND':'Indianapolis Colts',
    'JAX':'Jacksonville Jaguars',
    'KC':'Kansas City Chiefs',
    'LA':'Los Angeles Rams',
    'LAC':'Los Angeles Chargers',
    'LV':'Las Vegas Raiders',
    'MIA':'Miami Dolphins',
    'MIN':'Minnesota Vikings',
    'NE':'New England Patriots,
    'NO':'New Orleans Saints',
    'NYG':'New York Giants',
    'NYJ':'New York Jets',
    'PHI':'Philadelphia Eagles',
    'PIT':'Pittsburgh Steelers',
    'SEA':'Seattle Seahawks',
    'SF':'San Francisco 49ers',
    'TB':'Tampa Bay Buccaneers',
    'TEN':'Tennessee Titans',
    'WAS':'Washington Commanders'
}

SyntaxError: EOL while scanning string literal (2830134999.py, line 24)

In [59]:
data1 = pd.read_html('https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations')
# table = pd.DataFrame(data1[0], header=0)
table = table.rename(columns=table.iloc[0])
table = table.iloc[1:]
table


,Franchise,Official Team Abbreviation Codes,Commonly Used Abbreviations
1,Arizona Cardinals,ARZ,ARI
2,Atlanta Falcons,ATL,ATL
3,Baltimore Ravens,BLT,BAL
4,Buffalo Bills,BUF,BUF
5,Carolina Panthers,CAR,CAR
6,Chicago Bears,CHI,CHI
7,Cincinnati Bengals,CIN,CIN
8,Cleveland Browns,CLV,CLE
9,Dallas Cowboys,DAL,DAL
10,Denver Broncos,DEN,DEN


In [56]:
type(data1[0])

pandas.core.frame.DataFrame

In [62]:
ts = pd.read_html('https://www.nfl.com/standings/division/2022/REG')

In [63]:
ts[1]

,AFC NORTH,W,L,T,PCT,PF,PA,Net Pts,Home,Road,Div,Pct,Conf,Pct.1,Non-Conf,Strk,Last 5
0,Cincinnati Bengals xzBengals xz,12,4,0,0.750,418,322,96,6 - 1 - 0,6 - 3 - 0,3 - 3 - 0,0.5,8 - 3 - 0,0.727,4 - 1 - 0,8W,5 - 0 - 0
1,Baltimore Ravens xRavens x,10,7,0,0.588,350,315,35,5 - 3 - 0,5 - 4 - 0,3 - 3 - 0,0.5,6 - 6 - 0,0.500,4 - 1 - 0,2L,2 - 3 - 0
2,Pittsburgh SteelersSteelers,9,8,0,0.529,308,346,-38,4 - 4 - 0,5 - 4 - 0,3 - 3 - 0,0.5,5 - 7 - 0,0.417,4 - 1 - 0,4W,4 - 1 - 0
3,Cleveland BrownsBrowns,7,10,0,0.412,361,381,-20,4 - 4 - 0,3 - 6 - 0,3 - 3 - 0,0.5,4 - 8 - 0,0.333,3 - 2 - 0,1L,2 - 3 - 0


In [97]:
dt = pd.read_html('https://gist.github.com/cnizzardini/13d0a072adb35a0d5817')

In [98]:
tab = dt[0]
tab = tab[['Name', 'Abbreviation', 'Conference', 'Division']]
tab

,Name,Abbreviation,Conference,Division
0,Arizona Cardinals,ARI,NFC,West
1,Atlanta Falcons,ATL,NFC,South
2,Baltimore Ravens,BAL,AFC,North
3,Buffalo Bills,BUF,AFC,East
4,Carolina Panthers,CAR,NFC,South
5,Chicago Bears,CHI,NFC,North
6,Cincinnati Bengals,CIN,AFC,North
7,Cleveland Browns,CLE,AFC,North
8,Dallas Cowboys,DAL,NFC,East
9,Denver Broncos,DEN,AFC,West


In [100]:
tab = tab.sort_values(['Conference', 'Division','Abbreviation']).reset_index(drop=True)
# tab.loc[29, 'Abbreviation'] = 'LA'
tab.iloc[29,1] = 'LA'
tab

,Name,Abbreviation,Conference,Division
0,Buffalo Bills,BUF,AFC,East
1,Miami Dolphins,MIA,AFC,East
2,New England Patriots,NE,AFC,East
3,New York Jets,NYJ,AFC,East
4,Baltimore Ravens,BAL,AFC,North
5,Cincinnati Bengals,CIN,AFC,North
6,Cleveland Browns,CLE,AFC,North
7,Pittsburgh Steelers,PIT,AFC,North
8,Houston Texans,HOU,AFC,South
9,Indianapolis Colts,IND,AFC,South


In [101]:
tab['Div'] = tab['Conference'] + ' ' + tab['Division']
# tab['Conference'] + ' ' + tab['Division']


In [104]:
sorted(tab['Abbreviation']) == teams

True

In [105]:
tab

,Name,Abbreviation,Conference,Division,Div
0,Buffalo Bills,BUF,AFC,East,AFC East
1,Miami Dolphins,MIA,AFC,East,AFC East
2,New England Patriots,NE,AFC,East,AFC East
3,New York Jets,NYJ,AFC,East,AFC East
4,Baltimore Ravens,BAL,AFC,North,AFC North
5,Cincinnati Bengals,CIN,AFC,North,AFC North
6,Cleveland Browns,CLE,AFC,North,AFC North
7,Pittsburgh Steelers,PIT,AFC,North,AFC North
8,Houston Texans,HOU,AFC,South,AFC South
9,Indianapolis Colts,IND,AFC,South,AFC South


In [103]:
teams

['ARI',
 'ATL',
 'BAL',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GB',
 'HOU',
 'IND',
 'JAX',
 'KC',
 'LA',
 'LAC',
 'LV',
 'MIA',
 'MIN',
 'NE',
 'NO',
 'NYG',
 'NYJ',
 'PHI',
 'PIT',
 'SEA',
 'SF',
 'TB',
 'TEN',
 'WAS']

In [ ]:
#conference, divisions
AFCE = ['NYJ', 'NE', 'MIA', 'BUF']
AFCN = []
AFCS = []
AFCW = []
